In [6]:
import pandas as pd
import re

def extract_to_dataframe(text):
    try:
        # Define the regex pattern to extract instruction, input, and answer (as output)
        pattern = r'"instruction"\s*:\s*"([^"]+)"\s*,\s*"input"\s*:\s*"([^"]+)"\s*,\s*"answer"\s*:\s*"(.+)"'
        match = re.search(pattern, text, re.DOTALL)

        if match:
            # Return a list of the extracted components
            return [match.group(1), match.group(2), match.group(3)]
        else:
            return None
    except ValueError:
        return None

# Load your CSV file
file_path = '../raw_data/structured_instruct_input_output_pairs-3-10.csv'
data = pd.read_csv(file_path)

# Apply the function to extract data and create a DataFrame
data['extracted_data'] = data['output'].apply(extract_to_dataframe)
clean_data = data['extracted_data'].dropna().apply(pd.Series)
clean_data.columns = ['instruction', 'input', 'output']  # Rename the 'answer' column to 'output'

# Save the new DataFrame to a CSV file
clean_data.to_csv("structured_output_4.csv", index=False)
print(f"Structured data saved to 'structured_output.csv'. Total entries: {len(clean_data)}")


Structured data saved to 'structured_output.csv'. Total entries: 1977


--------------------

Not good practice but for now it's ok

In [7]:
import pandas as pd

# Paths to your CSV files
path1 = 'structured_output_1.csv'
path2 = 'structured_output_2.csv'
path3 = 'structured_output_3.csv'
path4 = 'structured_output_4.csv'
# Read the datasets
df1 = pd.read_csv(path1)
df2 = pd.read_csv(path2)
df3 = pd.read_csv(path3)
df4 = pd.read_csv(path4)
# Combine the datasets into one DataFrame
combined_df = pd.concat([df1, df2, df3, df4], ignore_index=True)

# Optionally, you can save the combined dataset to a new CSV file
combined_df.to_csv('combined_dataset_fine_tune_8k.csv', index=False)

print("Datasets combined. Total number of rows:", len(combined_df))


Datasets combined. Total number of rows: 7727


In [13]:
combined_df.loc[6000]['instruction']

'Provide the SQL query that retrieves the necessary data for a specific transaction type, status, and payment gateway type, aggregated by various account and product dimensions.'

In [18]:
import requests
import pandas as pd
import logging
import time
import os
from datetime import datetime as dt

API_URL = "https://openexchangerates.org/api/latest.json"
APP_ID = "ec2dc509f48a472fab8993b5363a0d92"

def fetch_exchange_rates():
    response = requests.get(f"{API_URL}?app_id={APP_ID}")
    if response.status_code == 200:
        return response.json()
    else:
        raise Exception("Failed to fetch exchange rates")


logging.basicConfig(level=logging.INFO)
logger = logging.getLogger()
logger.info("Starting exchange rate update...")
t0 = time.time()

# Fetch current exchange rates
data = fetch_exchange_rates()
rates = data["rates"]
timestamp = dt.utcfromtimestamp(data["timestamp"]).strftime('%Y-%m-%d %H:%M:%S')

# Create DataFrame
df = pd.DataFrame(rates.items(), columns=["currency", "rate"])
df["timestamp"] = timestamp

INFO:root:Starting exchange rate update...


In [22]:
rates

{'AED': 3.6729,
 'AFN': 72.303439,
 'ALL': 93.075798,
 'AMD': 387.584387,
 'ANG': 1.802059,
 'AOA': 834.06,
 'ARS': 885.2787,
 'AUD': 1.510157,
 'AWG': 1.8,
 'AZN': 1.7,
 'BAM': 1.811337,
 'BBD': 2,
 'BDT': 116.989513,
 'BGN': 1.809668,
 'BHD': 0.376846,
 'BIF': 2874.5,
 'BMD': 1,
 'BND': 1.353085,
 'BOB': 6.908975,
 'BRL': 5.1343,
 'BSD': 1,
 'BTC': 1.6187709e-05,
 'BTN': 83.504788,
 'BWP': 13.594751,
 'BYN': 3.272224,
 'BZD': 2.015457,
 'CAD': 1.365238,
 'CDF': 2800,
 'CHF': 0.90678,
 'CLF': 0.033311,
 'CLP': 919.13,
 'CNH': 7.241788,
 'CNY': 7.2337,
 'COP': 3874.57,
 'CRC': 512.246387,
 'CUC': 1,
 'CUP': 25.75,
 'CVE': 102.55,
 'CZK': 22.9263,
 'DJF': 176.5,
 'DKK': 6.896913,
 'DOP': 58.49475,
 'DZD': 134.340476,
 'EGP': 46.8507,
 'ERN': 15,
 'ETB': 57.424422,
 'EUR': 0.924415,
 'FJD': 2.2432,
 'FKP': 0.794417,
 'GBP': 0.794417,
 'GEL': 2.67125,
 'GGP': 0.794417,
 'GHS': 14.042048,
 'GIP': 0.794417,
 'GMD': 67.75,
 'GNF': 8592.5,
 'GTQ': 7.769283,
 'GYD': 209.187382,
 'HKD': 7.81212